<a href="https://colab.research.google.com/github/Edersao/Atmosf-ricos/blob/main/Precipitaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install netCDF4
!pip install basemap
!pip install datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 860.7/860.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 37.8 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 7.6 MB/s eta 0:00:00


In [ ]:
from netCDF4 import Dataset
import datetime as dt  # Incluiremos la librería datetime
import numpy as np
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/
import matplotlib.pyplot as plt
import matplotlib as pl
%matplotlib inline
from mpl_toolkits.basemap import Basemap, addcyclic, shiftgrid

## **Leer datos**

In [ ]:
arc_prec01 = '/content/drive/MyDrive/atmos/datos/Ixil/Ixil/Precipitacion/g4.timeAvgMap.TRMM_3B42_Daily_7_precipitation.20120101-20120131.90W_18N_86W_22N.nc'
prec201201 = Dataset(arc_prec01)

arc_prec08 = '/content/drive/MyDrive/atmos/datos/Ixil/Ixil/Precipitacion/g4.timeAvgMap.TRMM_3B42_Daily_7_precipitation.20120801-20120831.90W_18N_86W_22N.nc'
prec201208 = Dataset(arc_prec08)

prec201201.variables.keys()
prec201201.ncattrs()

['NCO',
 'nco_openmp_thread_number',
 'Conventions',
 'start_time',
 'end_time',
 'history',
 'userstartdate',
 'userenddate',
 'title',
 'plot_hint_title',
 'plot_hint_subtitle',
 'plot_hint_caption']

In [ ]:
prec201201.getncattr('start_time')
prec201201.getncattr('end_time')

'2012-02-01T01:29:59Z'

In [ ]:
lat = prec201201.variables['lat'][:]
lon = prec201201.variables['lon'][:]
preci01 = prec201201.variables['TRMM_3B42_Daily_7_precipitation']

In [ ]:
lat_inf = lat[0]
print(lat_inf)
lat_sup = lat[14]
print(lat_sup)
lon_inf = lon[0]
print(lon_inf)
lon_sup = lon[14]
print(lon_sup)

## **GRAFICAR**

In [ ]:
data_prec = preci08[:]
unidad = preci08.getncattr('units')
ix_lat = 21.1522
ix_lon = -89.4822

m = Basemap(projection='merc',llcrnrlat=18.375,urcrnrlat=21.875,\
            llcrnrlon=-90.375,urcrnrlon=-86.875,lat_ts=10,resolution = 'i')
m.drawcoastlines()
my_cmap=plt.cm.get_cmap('gist_rainbow')
my_cmap.set_under('w')
x0, y0 = m(ix_lon, ix_lat)
x, y = m(lon, lat)
m.plot(x0, y0, 'ok', markersize=5,color='w')
m.pcolormesh(x,y, data_prec)
cb = m.colorbar()
cb.set_label(unidad)
plt.text(x0, y0, ' Ixil', fontsize=12, color='w');
plt.title(prec201208.getncattr('plot_hint_title'))

## Leer todos los datos automáticamente y guardar la información

In [ ]:
from datetime import datetime, timedelta
import os
from netCDF4 import Dataset
import numpy as np
import glob
import csv
import pandas as pd

#df_ixil = pd.DataFrame(columns=['Fecha', 'Precipitacion_promedio'])
os.chdir("/content/drive/MyDrive/atmos/datos/Ixil/Ixil/Precipitacion")
archivos = sorted(glob.glob('g4.timeAvgMap.TRMM_3B42_Daily_7_precipitation*.nc'))

key_error = []
os_error = []
runtime_error = []

idx = pd.DatetimeIndex([""])
df_ixil=pd.DataFrame(columns=['Precipitacion_promedio'],index=idx)

Creamos dataframe donde se van a guardar los datos

In [ ]:
df_ixil

#new_row = {"a": 10, "b": 10}
#df2 = df.append(new_row, ignore_index=True)

#from datetime import date
#date(2002, 12, 4).isoformat()

,Precipitacion_promedio
NaT,NaN


In [ ]:
for arch in archivos:
    try:
        print(arch)
        data = Dataset(arch)
        fecha = datetime.strptime(data.getncattr('start_time'), '%Y-%m-%dT%H:%M:%SZ')
        date_formatted = datetime.strftime(fecha,'%d %b %Y')
        avg_prec = data.variables['TRMM_3B42_Daily_7_precipitation']
        new_row = {"Fecha": date_formatted, "Precipitacion_promedio": avg_prec}
        df1=pd.DataFrame(({"Precipitacion_promedio": avg_prec}),index=pd.DatetimeIndex([date_formatted]))
        df_ixil.append(df1)
    except KeyError:
        key_error.append(arch)
    except OSError:
        os_error.append(arch)
    except RuntimeError:
        runtime_error.append(arch)

In [ ]:
df_ixil

,Precipitacion_promedio
NaT,NaN


In [ ]:
date_formatted = datetime.strftime(fecha,'%d %b %Y')

In [ ]:
date_formatted

'01 Jan 2012'

In [ ]:
avg_prec[:]

Creación de DataFrame con indice temporal

In [ ]:
idx = pd.DatetimeIndex([""])
df=pd.DataFrame(columns=['Precipitacion_promedio'],index=idx)
df


tu código (mi versión):

In [ ]:
for arch in archivos:
    try:
        print(arch)
        data = Dataset(arch)
        fecha = datetime.strptime(data.getncattr('start_time'), '%Y-%m-%dT%H:%M:%SZ')
        date_formatted = datetime.strftime(fecha,'%d %b %Y')
        avg_prec = data.variables['TRMM_3B42_Daily_7_precipitation']
        new_row = {"Fecha": date_formatted, "Precipitacion_promedio": avg_prec}
        df1=pd.DataFrame(({"Precipitacion_promedio": avg_prec}),index=pd.DatetimeIndex([date_formatted]))
        df=df.append(df1)
    except KeyError:
        key_error.append(arch)
    except OSError:
        os_error.append(arch)
    except RuntimeError:
        runtime_error.append(arch)